# 0. 路徑

In [1]:
import threading # 因為等等要載入大量資料所以要multi thread
from multiprocessing import Queue

from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression, Lasso, Ridge, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, GradientBoostingClassifier,ExtraTreesClassifier

from xgboost import XGBClassifier
                        
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import accuracy_score

import datetime
import numpy as np
import pandas as pd
from numpy import loadtxt

import os
import argparse
from tqdm import tqdm
import matplotlib.pyplot as plt

plt.rcParams['font.family']='SimHei' #顯示中文

#特殊符號: | 

# 1A. 載入資料

In [2]:
# 載入資料

train = pd.read_csv('/home/jovyan/.kaggle/competitions/taiwan-aia-financefraud/train.csv', encoding = "utf-8", header=None)
test  = pd.read_csv('/home/jovyan/.kaggle/competitions/taiwan-aia-financefraud/test.csv', encoding = "utf-8", header=None)

# 將第一列改為 column names
colnm = train.iloc[0]
train.columns = colnm
train=train.iloc[1:]

colnm = test.iloc[0]
test.columns = colnm
test=test.iloc[1:]

print(train.shape)
print(test.shape)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,2,4,5,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,3,5,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(3817572, 10)
(2545047, 10)


In [3]:
train.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,Fraud
1,323,PAYMENT,9092.69,C749189758,0.0,0.0,M1939669337,0.0,0.0,0
2,164,CASH_IN,23468.75,C525009948,0.0,23468.75,C65353523,77300.17,53831.42,0
3,228,CASH_OUT,39688.11,C1311063915,0.0,0.0,C1581032686,168913.26,208601.37,0
4,229,CASH_OUT,179016.96,C700594269,103485.0,0.0,C90149178,2483034.47,2662051.42,0
5,709,CASH_OUT,77155.99,C646560157,143307.0,66151.01,C671337546,21213865.83,21291021.82,0


In [3]:
df = train

In [20]:
df = test

In [21]:
df.step = df.step.astype(float)

df = df.sort_values('step')
df['cnt']=1

print(df.head(3))
print(df.tail(3))

0             id  step     type   amount     nameOrig oldbalanceOrg  \
2120221  2120221   1.0    DEBIT  4444.35  C1184694332         10229   
59560      59560   1.0  PAYMENT   760.21   C288919635    1330599.59   
1471125  1471125   1.0  PAYMENT  2064.02  C1367612102   1.79863e+06   

0       newbalanceOrig     nameDest oldbalanceDest newbalanceDest  cnt  
2120221        5784.65   C611572016        26112.9        2608.64    1  
59560       1329839.38  M1572442646            0.0            0.0    1  
1471125    1.79657e+06  M1537371824              0              0    1  
0             id   step      type       amount     nameOrig oldbalanceOrg  \
292858    292858  743.0  TRANSFER       339682  C2013999242        339682   
815523    815523  743.0  TRANSFER  1.25882e+06  C1531301470   1.25882e+06   
1516576  1516576  743.0  CASH_OUT  6.31141e+06  C1162922333   6.31141e+06   

0       newbalanceOrig     nameDest oldbalanceDest newbalanceDest  cnt  
292858               0  C1850423904      

In [5]:
print(df.type.unique())

['PAYMENT' 'CASH_OUT' 'CASH_IN' 'DEBIT' 'TRANSFER']


In [22]:
df[(df['type']=='PAYMENT') & (df['Fraud']==1)].head(3)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,Fraud,cnt


In [23]:
df[(df['type']=='CASH_IN') & (df['Fraud']==1)].head(3)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,Fraud,cnt


In [24]:
df[(df['type']=='CASH_OUT') & (df['Fraud']==1)].head(3)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,Fraud,cnt
1842696,1.0,CASH_OUT,25071.5,C1275464847,25071.5,0,C1364913072,9083.76,34155.2,1,1
1316563,1.0,CASH_OUT,235239,C1499825229,235239,0,C2100440237,0,235239,1,1
860094,1.0,CASH_OUT,181,C840083671,181,0,C38997010,21182,0,1,1


In [25]:
df[(df['type']=='TRANSFER') & (df['Fraud']==1)].head(3)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,Fraud,cnt
1313746,1.0,TRANSFER,35063.6,C1364127192,35063.6,0,C1136419747,0,0,1,1
2424890,1.0,TRANSFER,2806,C1420196421,2806,0,C972765878,0,0,1,1
1810150,1.0,TRANSFER,1.27721e+06,C1334405552,1.27721e+06,0,C431687661,0,0,1,1


In [26]:
df[(df['type']=='DEBIT') & (df['Fraud']==1)].head(3)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,Fraud,cnt


# 1B 常用資料整理

In [97]:
#df['ColumnA'] = df['ColumnA'].astype(str)    # 因為值中有數字文字
#df.replace({'ColumnA':{'ValueA': 'ValueB'}}) # 值 誤植
'''
for dfB in full_data:  
    dfB['ColumnA'] = dfB['ColumnA'].fillna(dfA['ColumnA'].median())#刪除空值，用中位數代替  

df['columnB'] = pd.qcut(df['columnA'], 4)  #分為四等份
'''

# 1C 觀察原始資料

In [48]:
# 檢查原始資料 NaN 值
print(df.isnull().sum())

0
id                0
step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
cnt               0
dtype: int64


In [22]:
# 若加減法有str錯誤，則強迫全部回到numeric
df[['newbalanceOrig','oldbalanceOrg']]  = df[['newbalanceOrig','oldbalanceOrg']].apply(pd.to_numeric)
df[['newbalanceDest','oldbalanceDest']] = df[['newbalanceDest','oldbalanceDest']].apply(pd.to_numeric)
df[['amount']] = df[['amount']].apply(pd.to_numeric)

# 2A. Feature Engineering

In [23]:
# 逐筆計算差額
df['gap_Orig'] = round(df['newbalanceOrig'] - df['oldbalanceOrg'])
df['gap_Dest'] = round(df['newbalanceDest'] - df['oldbalanceDest'])
df['gap_amt_Orig']  = round(df['amount'] - abs(df['gap_Orig']))
df['gap_amt_Dest']  = round(df['amount'] - abs(df['gap_Dest']))
df['gap_Orig_Dest'] = round(abs(df['gap_Orig']) - abs(df['gap_Dest']))
df['gap_amt_Orig_Dest'] = round(abs(df['gap_amt_Orig']) - abs(df['gap_amt_Dest']))

# 時間
df['day']  = round(df['step']/24)
df['hour'] = df['step']%24

# Dest Acct 
df['M'] = 'M'
df['Dest_M'] = df.apply(lambda x: x['M'] in x['nameDest'], axis=1) 
df['Dest_M'] = df.Dest_M.astype(float)
#df['Fraud']  = df.Fraud.astype(float)
df.head()

,id,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,...,gap_Orig,gap_Dest,gap_amt_Orig,gap_amt_Dest,gap_Orig_Dest,gap_amt_Orig_Dest,day,hour,M,Dest_M
2120221,2120221,1.0,DEBIT,4444.35,C1184694332,10229.00,5784.65,C611572016,26112.94,2608.64,...,-4444.0,-23504.0,0.0,-19060.0,-19060.0,-19060.0,0.0,1.0,M,0.0
59560,59560,1.0,PAYMENT,760.21,C288919635,1330599.59,1329839.38,M1572442646,0.00,0.00,...,-760.0,0.0,0.0,760.0,760.0,-760.0,0.0,1.0,M,1.0
1471125,1471125,1.0,PAYMENT,2064.02,C1367612102,1798634.37,1796570.34,M1537371824,0.00,0.00,...,-2064.0,0.0,0.0,2064.0,2064.0,-2064.0,0.0,1.0,M,1.0
2533633,2533633,1.0,PAYMENT,10019.06,C500660587,5203.54,0.00,M1738784081,0.00,0.00,...,-5204.0,0.0,4815.0,10019.0,5204.0,-5204.0,0.0,1.0,M,1.0
1215059,1215059,1.0,PAYMENT,6490.69,C2054787417,14703.00,8212.31,M67811496,0.00,0.00,...,-6491.0,0.0,-0.0,6491.0,6491.0,-6491.0,0.0,1.0,M,1.0


In [42]:
g = df1.columns.to_series().groupby(df.dtypes).groups
g

{dtype('int64'): Index(['cnt'], dtype='object'),
 dtype('float64'): Index(['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
        'newbalanceDest', 'gap_Orig', 'gap_Dest', 'gap_amt_Orig',
        'gap_amt_Dest', 'gap_Orig_Dest', 'gap_amt_Orig_Dest', 'day', 'hour',
        'Dest_M'],
       dtype='object'),
 dtype('O'): Index(['id', 'type', 'nameOrig', 'nameDest', 'M'], dtype='object')}

In [25]:
# aggregate

# 同日同acct交易次數
DayOrig   = df.groupby(['nameOrig','day']).size()
DayDest   = df.groupby(['nameDest','day']).size()
DayOrigMax = DayOrig.groupby(level=0).max()
DayDestMax = DayDest.groupby(level=0).max()

# 同小時同acct交易次數
HurOrig   = df.groupby(['nameOrig','hour']).size()
HurDest   = df.groupby(['nameDest','hour']).size()
HurOrigMax = HurOrig.groupby(level=0).max()
HurDestMax = HurDest.groupby(level=0).max()


Orig   = df.groupby(['nameOrig']).size()
Dest   = df.groupby(['nameDest']).size()

TOr    = Orig.groupby(level=(0)).size() # 總共File被多少客人使用
TDe    = Dest.groupby(level=(0)).size() # 總共File被多少產品使用


In [26]:


df_Orig = pd.concat([DayOrigMax, HurOrigMax, TOr], axis=1)
df_Dest = pd.concat([DayDestMax, HurDestMax, TDe], axis=1)

df_Orig.columns = ['DayOrigMax', 'HurOrigMax', 'TOr']
df_Dest.columns = ['DayDestMax', 'HurDestMax', 'TDe']

df_Orig = pd.DataFrame(df_Orig).reset_index()
df_Dest = pd.DataFrame(df_Dest).reset_index()


In [40]:
df1 = pd.merge(df , df_Orig, on=['nameOrig'])
df1 = pd.merge(df1, df_Dest, on=['nameDest'])

In [19]:
df_train = df1

In [41]:
df_test = df1

# 4. 評估模型

In [29]:
# 分成 train & test

x_tr    = df_train.drop(['type', 'nameOrig', 'nameDest', 'M', 'cnt', 'Fraud'], axis=1)
y_tr    = df_train['Fraud']
x_te_ID = df_test['id']
x_te    = df_test.drop(['type', 'nameOrig', 'nameDest', 'M', 'cnt', 'id'], axis=1)

# 真正上傳時
x_train, y_train, x_test = x_tr, y_tr, x_te

# 訓練時
#x_train, x_test, y_train, y_test = train_test_split(x_tr, y_tr, test_size=0.2, random_state= 68, shuffle=False)


In [33]:
# XGBoost (train)

model = XGBClassifier() # n_estimators=100, max_depth=4, learning_rate=0.1
model.fit(x_train, y_train)
   
probs1 = model.predict_proba(x_train)
probs2 = model.predict_proba(x_test)

auc1   = metrics.roc_auc_score(y_train, [x[1] for x in probs1])
auc2   = metrics.roc_auc_score(y_test,  [x[1] for x in probs2])
print(auc1, auc2)


0.999556006115 0.999999737909


In [38]:
# XGBoost (真正上傳資料)

model = XGBClassifier() # n_estimators=100, max_depth=4, learning_rate=0.1
model.fit(x_train, y_train)
   
probs1 = model.predict_proba(x_train)
probs2 = model.predict_proba(x_test)

auc1   = metrics.roc_auc_score(y_train, [x[1] for x in probs1])
print(auc1)


TypeError: '>' not supported between instances of 'int' and 'str'

# 5A 組合模型 (Ensemble): KFold + Stacking 

# 5B1 給定跑模型的資料 

# 5B2 設定模型參數

# 5C Stacking 第一層

# 5D Stacking 第二層

# 5E 跳過stacking直接做KFold+XGB (一層而已)

# 6. 上傳結果

In [ ]:
KaggleSubmission = pd.DataFrame({ 'ID_t': x_te_ID,
                                  'Fraud': probs2[:,1] })
KaggleSubmission['ID'] = KaggleSubmission['ID_t'].astype(int)-1
KaggleSubmission = KaggleSubmission.drop(['ID_t'], axis=1)
KaggleSubmission.to_csv("KaggleSubmission.csv", index=False)

In [ ]:
KaggleSubmission